In [1]:
import pandas as pd
from enum import Enum
import unittest

In [2]:
class Result(Enum):
    WIN = 1
    LOSS = 0

In [3]:
card_denom_mapping = {
    'T': '10',
    'J': '11',
    'Q': '12',
    'K': '13',
    'A': '14'
}

In [4]:
def replace_card_denom(card_denom):
    if card_denom in card_denom_mapping:
        card_denom = card_denom_mapping[card_denom]
    return int(card_denom)

In [5]:
class PokerHand():
    def __init__(self, hand):
        self.str_hand = hand
        self.ranking, self.tie_breaker = self.rate_poker_hand(hand)
        
    def rate_poker_hand(self, hand):
        arr_hand = [[replace_card_denom(card[0]), card[1]] for card in hand.split()]
        df = pd.DataFrame(arr_hand, columns=['denomination','suit'])
        group_suit = df.groupby('suit').count()

        # If there is only one group then the five cards 
        # belong to an unique suit (it is a Flush)
        if group_suit.shape[0] == 1:
            
            # Sort by denomination to see if the cards
            # are in denomination sequence
            df = df.sort_values(['denomination'])

            # Difference between 
            flush = df['denomination'].diff()[1:].unique()

            # If 1 is the only value, the cards are in denomination sequence
            if flush.shape[0] == 1 and flush[0] == 1:
                # If the max card is A then we have a ROYAL STRAIGHT FLUSH
                if df['denomination'].max() == 14:
                    ranking=10
                # If the max card is different than A 
                # then we have a STRAIGHT FLUSH
                else:
                    ranking=9
            # The cards are not in denomination sequence so we have a FLUSH 
            else:
                ranking=6
        else:
            group_denom = df.groupby('denomination').count()

            if group_denom.shape[0] == 2:
                if int(group_denom.max()) == 4:
                    ranking=8
                else:
                    ranking=7
            elif group_denom.shape[0] == 3:
                if int(group_denom.max()) == 3:
                    ranking=4
                else:
                    ranking=3
            elif group_denom.shape[0] == 4:
                ranking=2
            else:
                df = df.sort_values(['denomination'])
                flush = df['denomination'].diff()[1:].unique()
                # If 1 is the only value, the cards are in denomination sequence
                # so we have a Straight
                if flush.shape[0] == 1 and flush[0] == 1:
                    ranking=5
                else:
                    ranking=1

        return ranking, df['denomination'].sort_values(ascending=False)
    
    def compare_with(self, otherHand):
        if self.ranking > otherHand.ranking:  
            return Result.WIN
        elif self.ranking < otherHand.ranking:
            return Result.LOSS
        else:
            for i in range(len(self.tie_breaker)):
                if self.tie_breaker[i] > otherHand.tie_breaker[i]:
                    return Result.WIN
                elif self.tie_breaker[i] < otherHand.tie_breaker[i]:
                    return Result.LOSS
            return None

In [6]:
class TestPokerHand(unittest.TestCase):

    def test_comparison(self):
        self.assertTrue(PokerHand("TC TH 5C 5H KH").compare_with(PokerHand("9C 9H 5C 5H AC")) == Result.WIN)
        self.assertTrue(PokerHand("TS TD KC JC 7C").compare_with(PokerHand("JS JC AS KC TD")) == Result.LOSS)
        self.assertTrue(PokerHand("7H 7C QC JS TS").compare_with(PokerHand("7D 7C JS TS 6D")) == Result.WIN)
        self.assertTrue(PokerHand("5S 5D 8C 7S 6H").compare_with(PokerHand("7D 7S 5S 5D JS")) == Result.LOSS)
        self.assertTrue(PokerHand("AS AD KD 7C 3D").compare_with(PokerHand("AD AH KD 7C 4S")) == Result.LOSS)
        self.assertTrue(PokerHand("TS JS QS KS AS").compare_with(PokerHand("AC AH AS AS KS")) == Result.WIN)
        self.assertTrue(PokerHand("TS JS QS KS AS").compare_with(PokerHand("TC JS QC KS AC")) == Result.WIN)
        self.assertTrue(PokerHand("TS JS QS KS AS").compare_with(PokerHand("QH QS QC AS 8H")) == Result.WIN)
        self.assertTrue(PokerHand("AC AH AS AS KS").compare_with(PokerHand("TC JS QC KS AC")) == Result.WIN)
        self.assertTrue(PokerHand("AC AH AS AS KS").compare_with(PokerHand("QH QS QC AS 8H")) == Result.WIN)
        self.assertTrue(PokerHand("TC JS QC KS AC").compare_with(PokerHand("QH QS QC AS 8H")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("JH JC JS JD TH")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("4H 5H 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("7C 8S 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("TS TH TD JH JD")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("4H 5H 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("7C 8S 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("TS TH TD JH JD")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("4H 5H 9H TH JH").compare_with(PokerHand("7C 8S 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("4H 5H 9H TH JH").compare_with(PokerHand("TS TH TD JH JD")) == Result.LOSS)
        self.assertTrue(PokerHand("4H 5H 9H TH JH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("7C 8S 9H TH JH").compare_with(PokerHand("TS TH TD JH JD")) == Result.LOSS)
        self.assertTrue(PokerHand("7C 8S 9H TH JH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("TS TH TD JH JD").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)

In [7]:
TestPokerHand().test_comparison()